In [1]:
from pprint import pprint
import random
import datetime

from IPython.core.debugger import set_trace

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
# Training helpers
def get_trainable(model_params):
    return (p for p in model_params if p.requires_grad)


def get_frozen(model_params):
    return (p for p in model_params if not p.requires_grad)


def all_trainable(model_params):
    return all(p.requires_grad for p in model_params)


def all_frozen(model_params):
    return all(not p.requires_grad for p in model_params)


def freeze_all(model_params):
    for param in model_params:
        param.requires_grad = False


# list(get_trainable(model.parameters()))
# list(get_frozen(model.parameters()))
# all_trainable(model.parameters())
# all_frozen(model.parameters())

In [5]:
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd, numpy as np
import os

from torchvision import transforms
import cv2

In [6]:
IMG_SIZE = 416
_mean = [0.485, 0.456, 0.406]
_std = [0.229, 0.224, 0.225]


train_trans = transforms.Compose([
    transforms.Resize(512),  # some images are pretty small
    transforms.RandomCrop(IMG_SIZE),
    transforms.RandomRotation([0.1, 0.90]),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(.3, .3, .3),
    transforms.ToTensor(),
])
val_trans = transforms.Compose([
    transforms.Resize(512),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
])

In [7]:
class Facedataset(Dataset):
    """Face label dataset."""
    def __init__(self, csv_path, transform = None, test = False):
        """
        Args:
            csv_path (string): path to csv file
            img_path (string): path to the folder where images are
            transform: pytorch transforms for transforms and tensor conversion
        """
        # Transforms
        self.to_tensor = transforms.ToTensor()
        self.transform = transform
        self.test = test
        # Resizing
        self.resize = transforms.Resize(IMG_SIZE)
        # Read the csv file
        self.data_info = pd.read_csv(csv_path)
        # First column contains the image paths
        self.image_arr = self.data_info['PATH'].values
        if self.test == False:
            # Second column is the labels
            self.label_arr = self.data_info['LABEL'].values
        # Calculate len
        self.data_len = len(self.data_info.index)

    def __getitem__(self, index):
        # Get image name from the pandas df
        single_image_name = self.image_arr[index]
        if single_image_name == 'PATH':
            single_image_name = self.image_arr[index+1]
        # Open image
        img_as_img = Image.open(single_image_name)
        
        if self.transform is not None:
            img_as_tensor = self.transform(img_as_img)
        else:
            img_as_img = self.resize(img_as_img)
            # Transform image to tensor
            img_as_tensor = self.to_tensor(img_as_img)
        
        if self.test == False:
            # Get label(class) of the image based on the cropped pandas column
            single_image_label = int(self.label_arr[index])
            #single_image_label = self.to_tensor(torch.from_numpy(np.array(self.label_arr[index])))
        
            return img_as_tensor, single_image_label
        else:
            return img_as_tensor
    
    def __len__(self):
        return self.data_len

In [8]:
train_ds = Facedataset("data/train.csv", transform = train_trans)
val_ds = Facedataset("data/val.csv", transform = val_trans)
test = Facedataset("data/test.csv", test = True, transform = val_trans)

BATCH_SIZE = 32
n_classes = 2

In [9]:
from torch.utils.data import DataLoader


train_dl = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=8,
)
val_dl = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=8,
)
test = DataLoader(
    test,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=8,
)

In [10]:
from torchvision import models

In [11]:
def get_model(n_classes=2):
    model = models.resnet50(pretrained=True)
    freeze_all(model.parameters())
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(100352, n_classes)
    return model


model = get_model().to(device)

In [12]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [13]:
criterion = nn.CrossEntropyLoss()

In [14]:
optimizer = torch.optim.Adam(
    get_trainable(model.parameters()),
    # model.fc.parameters(),
    lr=0.001,
    # momentum=0.9,
)

In [15]:
N_EPOCHS = 50

In [16]:
import utils

In [ ]:
for epoch in range(N_EPOCHS):
    print(f"Epoch {epoch+1}/{N_EPOCHS}")
    
    # Train
    model.train()  # IMPORTANT
    running_loss = 0.0
    correct = []
    for X, y in train_dl:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        # with torch.set_grad_enabled(True):
        y_ = model(X)
        loss = criterion(y_, y)

        loss.backward()
        optimizer.step()
        
        # Statistics
        # print(f"    batch loss: {loss.item():0.3f}")
        _, y_label_ = torch.max(y_, 1)
        
        correct.append(utils.balanced_accuracy_score(y, y_label_))
        
        running_loss += loss.item() * X.shape[0]
    
    print(f"  Train Loss: {running_loss / len(train_dl.dataset)}")
    print(f"  Train BAC:  {np.mean(correct)}")
    
    
    # Eval
    model.eval()  # IMPORTANT
    
    running_loss, correct = 0.0, 0
    correct = []
    
    y_true = []
    y_predict = []
    with torch.no_grad():  # IMPORTANT
        for X, y in val_dl:
            X, y = X.to(device), y.to(device)
                    
            y_ = model(X)
        
            _, y_label_ = torch.max(y_, 1)
            
            correct.append(utils.balanced_accuracy_score(y, y_label_))

            loss = criterion(y_, y)
            running_loss += loss.item() * X.shape[0]
    torch.save(model.state_dict(), 'data/weights/resnet_face_spoof_{}.weights'.format(epoch))
    
    print(f"  Valid Loss: {running_loss / len(val_dl.dataset)}")
    print(f"  Valid BAC:  {np.mean(correct)}")
    print()

Epoch 1/50
  Train Loss: 1.3720067830425995
  Train BAC:  0.8831775776783426


/home/denis/GitHub/python/Competion/Boost.face/utils.py:58: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


  Valid Loss: 5.451457432129374
  Valid BAC:  0.7077439024390243

Epoch 2/50
  Train Loss: 1.1957018347960662
  Train BAC:  0.9178023500094282
  Valid Loss: 1.4162635450618934
  Valid BAC:  0.8756707317073171

Epoch 3/50
  Train Loss: 0.9215596634634498
  Train BAC:  0.932811500251086
  Valid Loss: 1.3893146208876395
  Valid BAC:  0.8759654471544713

Epoch 4/50
  Train Loss: 1.3436152244414763
  Train BAC:  0.9240266622011966
  Valid Loss: 1.8024409708848859
  Valid BAC:  0.8825711382113821

Epoch 5/50
  Train Loss: 1.262171597339424
  Train BAC:  0.932393253931461
  Valid Loss: 3.1673844762688854
  Valid BAC:  0.8610975609756097

Epoch 6/50
  Train Loss: 1.2886015762937533
  Train BAC:  0.9352657321946289
  Valid Loss: 6.118743597867389
  Valid BAC:  0.7522865853658537

Epoch 7/50
  Train Loss: 1.2216531957767776
  Train BAC:  0.9404368694621673
  Valid Loss: 3.0720509336369246
  Valid BAC:  0.8352337398373983

Epoch 9/50
  Train Loss: 1.2892310433785537
  Train BAC:  0.94317278873689

In [ ]:
model.eval().cuda()
y_predict = []
with torch.no_grad():  # IMPORTANT
    for X in test:
        X = X.cuda()
                    
        y_ = model(X)
        
        _, y_label_ = torch.max(y_, 1)
        y_predict.append(np.array(y_label_))

In [ ]:
x = []
for i in y_predict:
    x.extend(list(i))
x = np.array(x)

In [ ]:
pred = pd.read_csv('data/test.csv')
pred['PATH'] = pred['PATH'].apply(lambda x: x.split('/')[2])

In [ ]:
pred['PRED'] = ''
pred['PRED'] = 1-x

In [ ]:
pred.columns = [0, 2]

In [ ]:
pred.to_csv('result1488.csv', index = False)

In [ ]:
pred[2].sum()

In [ ]:
pd.read_csv('data/sample2.csv')